For ease of processing, suppose that all transactions were made in dollars and execution time was in ms.

For testing purposes additional data was added (or existing data was changed). Sometimes it doesn't make sense :)

In [1]:
import pandas as pd
import numpy as np

d = {
    'id': [111], 
    'type': ['asset'], 
    'instrument': ['MSFT'], 
    'open_time': ['2019-01-23 12:33:11Z'], # fixed typo: 33 -> 23
    'open_price': [22.33], 
    'open_volume': [10000], 
    'open_order_type': ['limit'], 
    'tr_type': ['short'], 
    'open_comission': [7], 
    'open_exec_time': [3],
    
    'close_time': ['2019-01-23 14:33:11Z'],  # fixed typo: 33 -> 23 and changed time
    'close_price': [33.44], 
    'close_volume': [5000], 
    'close_order_type': ['limit'], 
    'close_comission': [3], 
    'open_exec_time': [4]
}

trade_history = pd.DataFrame(data=d)

#additional data
trade_history = pd.concat([trade_history]*3, ignore_index=True)
trade_history.at[2, 'id'] = 112
trade_history.at[1, 'type'] = 'bond'
trade_history.at[1, 'instrument'] = 'CVX'
trade_history.at[1, 'tr_type'] = 'long'
trade_history.at[1, 'open_order_type'] = 'market'
trade_history.at[1, 'open_time'] = '2019-01-21 14:33:11Z'
trade_history.at[1, 'close_time'] = '2019-01-22 14:33:11Z'
trade_history.at[1, 'close_price'] = '21.42'

trade_history[['open_time', 'close_time']] = trade_history[['open_time', 'close_time']].apply(pd.to_datetime)
trade_history

,id,type,instrument,open_time,open_price,open_volume,open_order_type,tr_type,open_comission,open_exec_time,close_time,close_price,close_volume,close_order_type,close_comission
0,111,asset,MSFT,2019-01-23 12:33:11+00:00,22.33,10000,limit,short,7,4,2019-01-23 14:33:11+00:00,33.44,5000,limit,3
1,111,bond,CVX,2019-01-21 14:33:11+00:00,22.33,10000,market,long,7,4,2019-01-22 14:33:11+00:00,21.42,5000,limit,3
2,112,asset,MSFT,2019-01-23 12:33:11+00:00,22.33,10000,limit,short,7,4,2019-01-23 14:33:11+00:00,33.44,5000,limit,3


In [2]:
d_e = {
    'start': ['2019-01-23 10:00:00Z', '2019-01-23 10:00:00Z'], 
    'end': ['2019-01-26 10:00:00Z', '2019-01-26 10:00:00Z'],
    'event_type': ['negative', 'expected'],
    'name': ['OPEC meeting', 'key rate change']
}

events = pd.DataFrame(data=d_e)
events[['start', 'end']] = events[['start', 'end']].apply(pd.to_datetime)
events

,start,end,event_type,name
0,2019-01-23 10:00:00+00:00,2019-01-26 10:00:00+00:00,negative,OPEC meeting
1,2019-01-23 10:00:00+00:00,2019-01-26 10:00:00+00:00,expected,key rate change


Assume we have data about sectors and their correlations:

In [3]:
d_s = {
    'instrument': ['MSFT', 'CVX'],
    'sector': ['technology', 'energy']
} 
sectors = pd.DataFrame(data=d_s).set_index('instrument')
sectors

,sector
instrument,
MSFT,technology
CVX,energy


In [4]:
d_cor = {
    'sector': ['technology', 'energy', 'financials'],
    'technology': [1, 0.37, 0.51], 
    'energy': [0.37, 1, 0.49],
    'financials': [0.51, 0.49, 1]
}

sector_corr = pd.DataFrame(data=d_cor).set_index('sector')
sector_corr = sector_corr.sort_values(by='sector').reindex(sorted(sector_corr.columns), axis=1)
sector_corr

,energy,financials,technology
sector,,,
energy,1.00,0.49,0.37
financials,0.49,1.00,0.51
technology,0.37,0.51,1.00


In [5]:
trade_history = pd.merge(trade_history, sectors, on=['instrument'])
one_hot = pd.get_dummies(trade_history['sector'])
trade_history = trade_history.join(one_hot)
trade_history

,id,type,instrument,open_time,open_price,open_volume,open_order_type,tr_type,open_comission,open_exec_time,close_time,close_price,close_volume,close_order_type,close_comission,sector,energy,technology
0,111,asset,MSFT,2019-01-23 12:33:11+00:00,22.33,10000,limit,short,7,4,2019-01-23 14:33:11+00:00,33.44,5000,limit,3,technology,0,1
1,112,asset,MSFT,2019-01-23 12:33:11+00:00,22.33,10000,limit,short,7,4,2019-01-23 14:33:11+00:00,33.44,5000,limit,3,technology,0,1
2,111,bond,CVX,2019-01-21 14:33:11+00:00,22.33,10000,market,long,7,4,2019-01-22 14:33:11+00:00,21.42,5000,limit,3,energy,1,0


In [6]:
trader = pd.DataFrame()

## Сумма инвестиций

In [7]:
trade_history['open_total_price'] = trade_history['open_price'] * trade_history['open_volume']
trader['investment_sum'] = trade_history.groupby('id')['open_total_price'].sum()


## Коэффициент окупаемости

In [8]:
trade_history['profit'] = 1 + (trade_history['close_price'] - trade_history['open_price'])/trade_history['open_price']
trader['avg_profit'] = trade_history.groupby('id')['profit'].mean()

## Интересы

In [9]:
sector_counts = trade_history.groupby('id')['sector'].apply(lambda x: x.value_counts()/x.count()).unstack(1, fill_value=0)
sector_names = sectors['sector'].values
for sector in sector_corr.columns:
    if sector not in sector_counts.columns:
        sector_counts[str(sector)] = 0.0
        
sector_counts = sector_counts.add_prefix('share_of_')

trader = trader.join(sector_counts)

# Склонность к риску

### Волатильность портфеля

In [10]:
sector_cols = [col for col in trader if col.startswith('share_of_')]
sector_cols = sorted([col for col in trader if col.startswith('share_of_')])

trader['volatility'] = trader[sector_cols].apply(lambda w: np.linalg.multi_dot((w.T, sector_corr, w)), axis=1)

### Доля акций

In [11]:
trader['asset_share'] = trade_history.groupby('id')['type'].apply(lambda x: (x=='asset').sum()/x.count())

### Доля коротких сделок

In [12]:
trader['short_share'] = trade_history.groupby('id')['tr_type'].apply(lambda x: (x=='short').sum()/x.count())

### Доля маркет ордеров

In [13]:
open_market = trade_history.groupby('id')['open_order_type'].apply(lambda x: (x=='market').sum())
all_count = trade_history.groupby('id')['open_order_type'].count()
close_market = trade_history.groupby('id')['close_order_type'].apply(lambda x: (x=='market').sum())
trader_market_share = (open_market + close_market)/(2 * all_count)
trader_market_share = trader_market_share.reset_index()
trader_market_share.rename(columns={trader_market_share.columns[-1]: 'market_share'}, inplace=True)
trader_market_share = trader_market_share.set_index('id')
trader = trader.join(trader_market_share)
# trader = pd.merge(trader, trader_market_share, on=['id']).set_index('id')

### Cредний срок удержания позиций

In [14]:
trade_history['retention_hours'] = (trade_history['close_time'] - trade_history['open_time']).astype('timedelta64[h]')
trader['retention_avg'] = trade_history.groupby('id')['retention_hours'].apply(lambda x: x.mean())

### Доля сделок, совершаемых во время негативных событий

In [15]:
trade_history['tmp'] = 1
events['tmp'] = 1
joined = pd.merge(trade_history, events, on=['tmp'])
joined = joined.drop('tmp', axis=1)
joined = joined[joined['event_type'] == 'negative'].reset_index()
joined['negative_share'] = (((joined['open_time'] >= joined['start']) & (joined['open_time'] <= joined['end'])) | 
                         ((joined['close_time'] >= joined['start']) & (joined['close_time'] <= joined['end'])))
is_negative = joined.groupby(['id'])['negative_share'].mean().reset_index().set_index('id')
trader = trader.join(is_negative)

# Final Trader Features

In [16]:
trader

,investment_sum,avg_profit,share_of_energy,share_of_technology,share_of_financials,volatility,asset_share,short_share,market_share,retention_avg,negative_share
id,,,,,,,,,,,
111,446600.0,1.228392,0.5,0.5,0.0,0.685,0.5,0.5,0.25,13.0,0.5
112,223300.0,1.497537,0.0,1.0,0.0,1.000,1.0,1.0,0.00,2.0,1.0
